<a href="https://colab.research.google.com/github/pachterlab/HCP_2024/blob/main/analysis_Barr_GC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE201nnn/GSE201516/suppl/GSE201516%5FDataS1.csv.gz

In [1]:
!mkdir -p results
!mkdir -p figs

In [ ]:
import numpy as np
import pandas as pd

# plotting
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('GSE201516_DataS1.csv.gz')

In [ ]:
label_fs = 17
line_color = 'black'

def sub_make_plots(P,H,ax,
               axis_fs = 12,title_fs = 15,
               s = 2, alpha = 0.1,color='gray'):

  delta = P - H
  scaled_theta = (2/np.pi) * np.arctan( H / delta )
  R = np.sqrt( delta**2 + H**2)
  # get cis prop
  cis_prop_reordered = scaled_theta-0.5
  cis_prop_reordered[cis_prop_reordered<=-1.0] = (cis_prop_reordered[cis_prop_reordered<=-1.0]) + 2.0

  ax.scatter(cis_prop_reordered, P ,s = s, alpha = alpha,color=color)
  ax.set_ylabel('$X_{12}$',fontsize=18)
  ax.axhline(0, color='black', linewidth=2, alpha=0.7, linestyle='--')
  ax.tick_params(axis='both', labelsize=label_fs)
  ax.tick_params(bottom=False)
  ax.grid(which='both', alpha=0.5, linewidth=0.5)
  ax.minorticks_on()

## Newly calculated vs. reported proportion $cis$
Plot newly calculated proportion *cis* versus the reported proportion *cis* from Barr *et al.*

In [ ]:
# calculate proportion cis

P = df['dream.logFC'].values.flatten()
H = df['hybrid.logFC'].values.flatten()


delta = P - H
scaled_theta = (2/np.pi) * np.arctan( H / delta )
R = np.sqrt( delta**2 + H**2)

df['P'] = P
df['H'] = H
df['R'] = R
df['scaled_theta'] = scaled_theta
df['cisprop_ours'] = np.abs(scaled_theta)


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5.4,5))
cis_prop = df['cisprop'].values.flatten()
scaled_theta_all = np.abs(df['scaled_theta'].values.flatten())
ax.scatter(cis_prop,scaled_theta_all, s = 4, color = 'darkred')
ax.set_xlabel('Proportion $cis$ (Barr $et$ $al.)$',fontsize = 18)
ax.set_ylabel('Proportion $cis$',fontsize = 18)
x_min, x_max = ax.get_xlim()
y_min, y_max = ax.get_ylim()
# Determine the diagonal points for the line y = x
diag_min = min(x_min, y_min)
diag_max = max(x_max, y_max)
ax.plot([0, diag_max], [0, diag_max], linewidth = 3.0, color='black',alpha=0.7,linestyle='--',
      label = 'y = x')
# Set x and y axis limits
ax.set_xlim(0, 1.02)
ax.set_ylim(0, 1.02)

ax.tick_params(axis='both',labelsize=label_fs)
ax.spines[['right', 'top']].set_visible(False)
ax.grid(which='both')
plt.legend(fontsize=18)
plt.tight_layout()
plt.savefig('./figs/fig3A',bbox_inches='tight',dpi=400)

In [ ]:
# FC difference

fig,ax = plt.subplots(1,1,figsize=(5.4,5))
cis_prop = df['cisprop'].values.flatten()
scaled_theta_all = np.abs(df['scaled_theta'].values.flatten())
ax.scatter(cis_prop,cis_prop/scaled_theta_all,
           s = 4, color = 'darkred')
ax.set_ylabel('Ratio',fontsize = 18)
ax.set_xlabel('Proportion $cis$ (Barr $et$ $al.)$',fontsize = 18)

# Set x and y axis limits
# ax.set_xlim(0, 1.02)
# ax.set_ylim(0, 1.02)

ax.tick_params(axis='both',labelsize=label_fs)
ax.grid(which='both')
# ax.minorticks_on()
ax.spines[['right', 'top']].set_visible(False)
# plt.legend(fontsize=18)
plt.tight_layout()
ax.axhline(1.0,linewidth = 3.0, color='black',alpha=0.7,linestyle='--')
plt.savefig('./figs/barr_cis_fc',bbox_inches='tight',dpi=400)

## Variance and mean of proportion $cis$ across cell types

In [ ]:
var_cis_prop_df =  pd.DataFrame(df.groupby(['gene'])['cisprop_ours'].apply(lambda g: g.var(skipna=False)))
mean_cis_prop_df = pd.DataFrame(df.groupby(['gene'])['cisprop_ours'].apply(lambda g: g.mean(skipna=False)))

# gene thresholds: make sure they are not ambiguous in at least 35/72 cell types
thresh = 0
df_reg = df
number_not_null = df_reg.groupby(['gene']).count()['cell.type']
genes_not_null = number_not_null[number_not_null>thresh].index

var_cis_prop_df = var_cis_prop_df.loc[genes_not_null,:].dropna()
mean_cis_prop_df = mean_cis_prop_df.loc[var_cis_prop_df.index,:]

In [ ]:
cis_thresh = 0.11
high_var_genes = var_cis_prop_df[var_cis_prop_df['cisprop_ours']>cis_thresh].index
high_var_genes

In [ ]:
axis_fs = 25
title_fs = 25
label_fs = 20
s = 45
alpha = .7
line_color = 'black'
fig,ax = plt.subplots(1,4,figsize=(20,5))


axis_width = 4.0
line_alpha = 0.8

gene_names = ['Mageh1','Bloc1s1','Dnmt1','Ppil4']

for i,gene in enumerate(['MAGEH1','BLOC1S1','DNMT1','PPIL4']):

  df_gene = df[df['gene'].isin([gene])]
  P = df_gene['P'].values.flatten()
  H = df_gene['H'].values.flatten()


  sub_make_plots(P,H,ax=ax[i],
               axis_fs = axis_fs,title_fs = title_fs,
               s = s, alpha = alpha, color = 'black')


  ax[i].set_ylabel('$R_P$',fontsize=axis_fs)
  ax[i].set_title(f'${gene_names[i]}$',fontsize=title_fs)

  ax[i].set_ylim(-2.2,5.27)
  ax[i].set_xlim(-1.05,1.05)
  ax[i].tick_params(which='both',labelsize=label_fs)

  ax[i].axvline(0.5, color='orangered', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[i].axvline(-0.5, color='darkblue', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[i].axvline(0.0, color='skyblue', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[i].axvline(-1, color='forestgreen', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[i].axvline(1, color='forestgreen', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[i].axhline(0, color='gray', linewidth=axis_width, alpha = line_alpha,zorder=0)
  ax[i].axhline(0, color='black', linewidth=2, alpha=0.7, linestyle='-')

  ax[i].tick_params(axis='both', labelsize=label_fs)
  ax[i].tick_params(bottom=False)
  ax[i].grid(which='both', alpha=0.5, linewidth=0.5)
  ax[i].minorticks_on()

plt.tight_layout()
plt.savefig('./figs/barr_highvar_reg.png',dpi=400,bbox_inches='tight')

In [ ]:
gene_colors = ['gold','plum','red','magenta']

In [ ]:
cell_types = df['cell.type'].unique()

n = 1

fig,ax = plt.subplots(6,3,figsize = (3*3,6*3) )
alpha = 0.3
s = 2.0
axis_fs = 14
title_fs = 14
label_fs = 14
for i,ct in enumerate(cell_types[:18]):

  ind1 = i%6
  ind2 = int( np.floor(i/6))

  df_gene = df[df['cell.type'].isin([ct])]
  P = df_gene['P'].values.flatten()
  H = df_gene['H'].values.flatten()


  sub_make_plots(P,H,ax=ax[ind1,ind2],
               axis_fs = axis_fs,title_fs = title_fs,
               s = s, alpha = alpha, color = 'black')


    # add genes

  for g,gene in enumerate(['MAGEH1','BLOC1S1','DNMT1','PPIL4']):
      df_gene_ = df_gene[df_gene['gene'].isin([gene])]
      P = df_gene_['P'].values.flatten()
      H = df_gene_['H'].values.flatten()


      sub_make_plots(P,H,ax=ax[ind1,ind2],
                   axis_fs = axis_fs,title_fs = title_fs,
                   s = 30, alpha = 1.0, color = gene_colors[g],)

  ax[ind1,ind2].set_ylabel('$R_P$',fontsize=axis_fs)
  ax[ind1,ind2].set_xlabel('Proportion $cis$',fontsize=axis_fs)
  if ct == 'Syncytiotrophoblasts and villous cytotrophoblasts':
        ct = 'Syncytiotrophoblasts and \n villous cytotrophoblasts'
  ax[ind1,ind2].set_title(f'{ct}',fontsize=title_fs)

  ax[ind1,ind2].set_ylim(-9.0,8.0)
  ax[ind1,ind2].set_xlim(-1.05,1.05)
  ax[ind1,ind2].tick_params(which='both',labelsize=label_fs)

  ax[ind1,ind2].axvline(0.5, color='orangered', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[ind1,ind2].axvline(-0.5, color='darkblue', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[ind1,ind2].axvline(0.0, color='skyblue', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[ind1,ind2].axvline(-1, color='forestgreen', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[ind1,ind2].axvline(1, color='forestgreen', linewidth=axis_width, alpha=line_alpha,zorder=0)
  ax[ind1,ind2].axhline(0, color='gray', linewidth=axis_width, alpha = line_alpha,zorder=0)
  ax[ind1,ind2].axhline(0, color='black', linewidth=2, alpha=0.7, linestyle='-')

  ax[ind1,ind2].tick_params(axis='both', labelsize=label_fs)
  ax[ind1,ind2].tick_params(bottom=False)
  ax[ind1,ind2].grid(which='both', alpha=0.5, linewidth=0.5)
  ax[ind1,ind2].minorticks_on()
  ax[ind1,ind2].set_xticks( [-1,-0.5,0.0,0.5,1.0], labels = [0.5,0.0,0.5,1.0,0.5])

plt.tight_layout()
plt.savefig(f'./figs/barr_celltypes_{n}.png',dpi=400,bbox_inches='tight')


